In [59]:
"""
Lexical functions' id :
Oper1: ls:fr:lf:168
Oper2: ls:fr:lf:190
Oper3: ls:fr:lf:193
Func0: ls:fr:lf:195
Func1: ls:fr:lf:213
Func2: ls:fr:lf:219
Func3: ls:fr:lf:222
Labor12: ls:fr:lf:224
Labor13: ls:fr:lf:961
Labpr21: ls:fr:lf:225

"""

"\nLexical functions' id :\nOper1: ls:fr:lf:168\nOper2: ls:fr:lf:190\nOper3: ls:fr:lf:193\nFunc0: ls:fr:lf:195\nFunc1: ls:fr:lf:213\nFunc2: ls:fr:lf:219\nFunc3: ls:fr:lf:222\nLabor12:\nLabor13:\n\n"

In [34]:
import pandas as pd
import os 

In [16]:
"""
current_dir = os.path.dirname("parsing_lf_verb.ipynb")
csv_path = os.path.join(current_dir, '..', '..', 'lexical-system-fr', 'ls-fr-V3', '15-lslf-rel.csv')
df = pd.read_csv(csv_path, delimiter='\t')
filtered_df = df[df['lf'] == 'ls:fr:lf:225'] # Oper / Func / Labor
result = filtered_df[['source', 'target']]
output_path = os.path.join(current_dir, '..', 'result_Labor21.csv')
result.to_csv(output_path, index=False)
print(result)

"""

                 source            target
402    ls:fr:node:26291  ls:fr:node:43932
5394   ls:fr:node:27309  ls:fr:node:43932
5528   ls:fr:node:27348  ls:fr:node:26434
17679  ls:fr:node:30858  ls:fr:node:43932


In [60]:
# Load csv file 
df = pd.read_csv('../../lexical-system-fr/ls-fr-V3/15-lslf-rel.csv', delimiter='\t')  
nodes_df = pd.read_csv('../../lexical-system-fr/ls-fr-V3/01-lsnodes.csv', delimiter='\t')

# Filtrer les lignes où la colonne 'lf' est égale à 'id de la lf'
filtered_df = df[df['lf'] == 'ls:fr:lf:168'] # Oper / Func / Labor

# Sélectionner les colonnes 'source' et 'target'
result = filtered_df[['source', 'target']]
nodes_df['lexname_cleaned'] = nodes_df['lexname'].str.extract(r'>\s*(.+?)\s*<')
id_to_lexname = pd.Series(nodes_df.lexname_cleaned.values, index=nodes_df.id).to_dict()
# Appliquer les correspondances aux colonnes 'source' et 'target' de result
result.loc[:, 'source_lexname'] = result['source'].map(id_to_lexname)
result.loc[:, 'target_lexname'] = result['target'].map(id_to_lexname)
#result.to_csv('../support_verb/result_Labor21.csv', index=False)
# Afficher les résultats
print(result)
# Sauvegarder le résultat enrichi dans un nouveau fichier CSV
output_file = '../support_verb/lf_source_target/Oper1.csv'
result.to_csv(output_file, index=False)

print("Les données enrichies ont été sauvegardées dans", output_file)




                 source            target source_lexname target_lexname
332    ls:fr:node:26269  ls:fr:node:50447       allergie       souffrir
395    ls:fr:node:26291  ls:fr:node:34765            ami           être
418    ls:fr:node:26292  ls:fr:node:56691          amour          avoir
419    ls:fr:node:26292  ls:fr:node:27370          amour       éprouver
420    ls:fr:node:26292  ls:fr:node:56763          amour         porter
...                 ...               ...            ...            ...
64600  ls:fr:node:56701  ls:fr:node:50447         ictère       souffrir
64602  ls:fr:node:56702  ls:fr:node:56691     étonnement          avoir
64603  ls:fr:node:56702  ls:fr:node:27370     étonnement       éprouver
64604  ls:fr:node:56702  ls:fr:node:29223     étonnement      ressentir
64686  ls:fr:node:56760  ls:fr:node:56763       jugement         porter

[813 rows x 4 columns]
Les données enrichies ont été sauvegardées dans ../support_verb/lf_source_target/Oper1.csv


C:\Users\Etudiant\AppData\Local\Temp\ipykernel_18408\306590752.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.loc[:, 'source_lexname'] = result['source'].map(id_to_lexname)
C:\Users\Etudiant\AppData\Local\Temp\ipykernel_18408\306590752.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.loc[:, 'target_lexname'] = result['target'].map(id_to_lexname)


In [28]:
from transformers import pipeline, CamembertTokenizer, CamembertModel

def generate_text(source, verb):
    # Initialiser le tokenizer et le modèle pour CamemBERT
    tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
    model = CamembertModel.from_pretrained('camembert-base')
    
    # Charger le pipeline de génération de texte en utilisant CamemBERT
    generator = pipeline('text-generation', model='camembert-base')

    # Créer un prompt avec le nom et le verbe
    prompt = f"Écrire une phrase utilisant le nom '{source}' et le verbe '{verb}':"

    # Générer la réponse à partir du prompt en utilisant les tokens d'EOS comme pad_token_id
    responses = generator(prompt, max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

    # Prendre la première réponse générée
    generated_text = responses[0]['generated_text']
    return generated_text.strip()

# Exemple d'utilisation
source_lexname = "haine"
target_lexname = "inspirer"
sentence = generate_text(source_lexname, target_lexname)
print(sentence)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
If you want to use `CamembertLMHeadModel` as a standalone, add `is_decoder=True.`
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Écrire une phrase utilisant le nom 'haine' et le verbe 'inspirer':


In [31]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

def generate_text(source, verb):
    # Charger le tokenizer et le modèle GPT-2
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    # Créer un pipeline de génération de texte avec GPT-2
    text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

    # Créer un prompt avec le nom et le verbe
    prompt = f"Écrire une phrase en utilisant le nom '{source}' et le verbe '{verb}':"

    # Générer la réponse à partir du prompt
    responses = text_generator(prompt, max_length=100, num_return_sequences=1)

    # Prendre la première réponse générée
    generated_text = responses[0]['generated_text']
    return generated_text.strip()

# Exemple d'utilisation
source_lexname = "haine"
target_lexname = "inspirer"
sentence = generate_text(source_lexname, target_lexname)
print(sentence)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Écrire une phrase en utilisant le nom 'haine' et le verbe 'inspirer': il se prêté se trouver les journaux que saur une personne de l'infrage à nût. Le voiture est même, il a jût, le verbe.

Efrit-l-décor,

'Décor-l-décor le faç
